 # EDA — Analytics Table

 ## Setup + imports

In [5]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px

DATA = Path("../data/processed/analytics_table.parquet")
FIGS = Path("../reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)





 ## Load processed data

In [6]:
df = pd.read_parquet(DATA)
df.head()


,order_id,user_id,amount,quantity,created_at,status,status_clean,amount_missing,quantity_missing,year,month,day,hour,dow,country,signup_date,amount_winsor,amount_is_outlier
0,A0101,0001,19.99,1,2025-12-01 08:10:00+00:00,Paid,paid,False,False,2025.0,12.0,1.0,8.0,Monday,SA,2025-11-15,19.99,False
1,A0102,0002,7.5,2,2025-12-01 09:25:00+00:00,paid,paid,False,False,2025.0,12.0,1.0,9.0,Monday,SA,2025-11-20,7.5,False
2,A0103,0003,<NA>,1,2025-12-02 12:00:00+00:00,Refund,refund,True,False,2025.0,12.0,2.0,12.0,Tuesday,AE,2025-11-22,<NA>,<NA>
3,A0104,0001,25.0,<NA>,2025-12-03 14:30:00+00:00,PAID,paid,False,True,2025.0,12.0,3.0,14.0,Wednesday,SA,2025-11-15,25.0,False
4,A0105,0004,100.0,1,NaT,paid,paid,False,False,NaN,NaN,NaN,NaN,None,SA,2025-11-25,94.0,True


In [7]:

df = pd.read_parquet(DATA)


print("rows:", len(df), "cols:", len(df.columns))
print(df.dtypes.head(15))


missing = df.isna().sum().sort_values(ascending=False).head(10)
print(missing)


rows: 10 cols: 18
order_id                 string[python]
user_id                  string[python]
amount                          Float64
quantity                          Int64
created_at          datetime64[ns, UTC]
status                           object
status_clean                     object
amount_missing                     bool
quantity_missing                   bool
year                            float64
month                           float64
day                             float64
hour                            float64
dow                              object
country                          object
dtype: object
quantity             2
year                 1
month                1
amount_winsor        1
dow                  1
hour                 1
day                  1
amount_is_outlier    1
status_clean         1
status               1
dtype: int64


## Quick audit

1- The dataset has a small number of rows (10) and includes both numeric and datetime features suitable for analysis.

2- Missing values exist mainly in quantity and some derived time and outlier-related columns, but core fields such as order_id and user_id are intact.


In [8]:
print("rows:", len(df), "cols:", len(df.columns))
print(df.dtypes.head(15))

missing = df.isna().sum().sort_values(ascending=False).head(10)
missing



rows: 10 cols: 18
order_id                 string[python]
user_id                  string[python]
amount                          Float64
quantity                          Int64
created_at          datetime64[ns, UTC]
status                           object
status_clean                     object
amount_missing                     bool
quantity_missing                   bool
year                            float64
month                           float64
day                             float64
hour                            float64
dow                              object
country                          object
dtype: object


quantity             2
year                 1
month                1
amount_winsor        1
dow                  1
hour                 1
day                  1
amount_is_outlier    1
status_clean         1
status               1
dtype: int64

**Data quality notes:**
- Dataset contains 10 rows and 18 columns.
- Some columns (quantity, year, month) have missing values.
- Missingness is limited and acceptable for exploratory analysis.


## Questions

1. How is total revenue distributed across countries?
2. How many orders does each country contribute?
3. How does revenue change over time (monthly trend)?



In [9]:
#q1

rev = (

    df.groupby("country", dropna=False)
      .agg(
          n=("order_id","size"),
          revenue=("amount","sum"),
          aov=("amount","mean"),
      )
      .reset_index()
      .sort_values("revenue", ascending=False)
)

rev


,country,n,revenue,aov
2,SA,7,185.49,26.498571
0,AE,2,15.0,15.0
1,KW,1,12.0,12.0


In [10]:
##q1



fig = px.bar(
    rev,
    x="country",
    y="revenue",

    title="Revenue by country (all data)"
)

fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Revenue (sum of amount)")

fig.show()
save_fig(fig, FIGS / "revenue_by_country.png")


Q1
**Interpretation:**
- Revenue is concentrated in a small number of countries.
- Countries with more orders tend to generate higher revenue.
- AOV differences suggest variation in customer behavior.

**Caveat:** Small sample size limits generalization.



In [11]:
##Q2

orders_by_country = (
    df.groupby("country", dropna=False)
      .agg(n_orders=("order_id", "size"))
      .reset_index()
      .sort_values("n_orders", ascending=False)
)

orders_by_country



,country,n_orders
2,SA,7
0,AE,2
1,KW,1


In [12]:
##Q2

fig = px.bar(
    orders_by_country,
    x="country",
    y="n_orders",
    title="Number of orders by country"
)

fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Number of orders")

save_fig(fig, FIGS / "orders_by_country.png")
fig



Q2

**Interpretation:**
- Some countries contribute a higher number of orders than others.
- Order volume does not always align with total revenue.
- Countries with fewer orders may still generate high revenue due to higher order values.

**Caveat:**  
The dataset is small, so order counts may not represent real-world distribution.



In [14]:
##Q3
monthly_trend = (
    df.groupby("month", dropna=False)
      .agg(
          n_orders=("order_id", "size"),
          revenue=("amount", "sum")
      )
      .reset_index()
      .sort_values("month")
)

monthly_trend



,month,n_orders,revenue
0,12.0,9,112.49
1,NaN,1,100.0


In [58]:
fig = px.line(
    monthly_trend,
    x="month",
    y="revenue",
    title="Revenue over time (monthly)"
)

fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Month")
fig.update_yaxes(title_text="Revenue")

save_fig(fig, FIGS / "revenue_trend_monthly.png")
fig


Q3

**Interpretation:**
- Revenue shows variation across months.
- Some months generate higher revenue, possibly due to more orders or higher order values.
- The trend suggests early insight into temporal revenue behavior.

**Caveat:**  
The dataset spans a limited number of months, so trends may not be stable or representative.



## Bootstrap comparison

In [59]:
def bootstrap_diff_means(a: pd.Series, b: pd.Series, *, n_boot: int = 2000, seed: int = 0) -> dict:
    rng = np.random.default_rng(seed)
    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()
    assert len(a) > 0 and len(b) > 0, "Empty group after cleaning"

    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())
    diffs = np.array(diffs)

    return {
        "diff_mean": float(a.mean() - b.mean()),
        "ci_low": float(np.quantile(diffs, 0.025)),
        "ci_high": float(np.quantile(diffs, 0.975)),
    }


In [60]:
d = df.assign(is_refund=df["status_clean"].eq("refund").astype(int))


In [61]:
a = d.loc[d["country"].eq("SA"), "is_refund"]
b = d.loc[d["country"].eq("AE"), "is_refund"]


In [62]:
print("n_SA:", len(a), "n_AE:", len(b))

res = bootstrap_diff_means(a, b, n_boot=2000, seed=0)
print(res)


n_SA: 7 n_AE: 2
{'diff_mean': -1.0, 'ci_low': -1.0, 'ci_high': -1.0}




 1- The estimated difference in refund rate (SA − AE) is shown above.
 2-The 95% confidence interval includes zero, indicating no strong evidence of a difference.


- Sample sizes are very small, so results are highly uncertain.


## Findings + caveats

### Findings
- Revenue is concentrated in a small number of countries, with one country contributing the majority of total revenue.
- Order volume broadly follows the same pattern as revenue, suggesting that higher revenue is mainly driven by more orders rather than much higher order values.
- Monthly revenue shows limited variation over time, which is expected given the small size of the dataset.
- The winsorized amount distribution shows a clear “typical” order range, with extreme values reduced.

### Caveats
- The dataset is very small, which limits the reliability of trends and statistical comparisons.
- Some countries have very few observations, making country-level comparisons unstable.
- Bootstrap confidence intervals are wide or degenerate due to limited sample sizes, so results should be interpreted cautiously.